In [1]:
import tensorflow as tf
import os

In [2]:
import numpy as np
import pandas as pd

In [3]:
import matplotlib.pyplot as plt

In [4]:
filepath = "h5s/data02/"

In [ ]:
model = tf.keras.models.load_model(filepath)

In [5]:
#no crop function
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
    img = tf.io.decode_jpeg(a)
    #*********
    #img = tf.image.adjust_contrast(img, 2)
    #*********
    img = tf.image.convert_image_dtype(img, dtype=tf.float16)/255.
    img = tf.image.per_image_standardization(img)
    return img


In [6]:

CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

In [7]:
def parseResult(RS):
    for j in range(len(RS[0])):
        PredChr=''
        for i in range(6):
            PredChr+= CLASS_NAMES[(np.where(RS[i][j] == RS[i][j].max())[0][0])]
        if j == 0:
            LB = [PredChr]
        else:
            LB.append(PredChr)      
    return LB
     

In [8]:
model_paths=["railway_d2e-1.hdf5","CNN_GRU_3_fine2.hdf5","CNN_GRU_4.hdf5","CNN_GRU_5.hdf5","CNN+GRU.hdf5","CNN_GRU_2.hdf5"] 

In [9]:
def ensemble_chr(model_paths,img_test):
    '''return the most probable characters for given models'''
    models = []
    for m in model_paths:
        models.append(tf.keras.models.load_model(filepath+m))
    result = []
    for m in models:
        result.append(m.predict(img_test))
    r = np.array(result)
    a = np.argmax(r,axis=3)
    a = a.transpose()
    a_p = np.max(r,axis=3)
    a_p = a_p.transpose()
    #return the most probable character index
    vote_outcome=[]
    for j in range(len(a)):
        chr = ''
        for i,p in zip(a[j],a_p[j]):
            if(all(i==i[0])):
                chr+=(CLASS_NAMES[i[0]])
            else:
                maj = np.bincount(i).argmax()   #i[np.argmax(p)]
                chr+=(CLASS_NAMES[maj])           
        vote_outcome.append(chr)
    
    return vote_outcome

In [ ]:
#01
val_dir_data = os.path.abspath(os.getcwd())+"\\test\\data01_test"
#validation data no crop
val_dir = tf.data.Dataset.list_files(val_dir_data+'*.jpg',shuffle=False) 
val_data = val_dir.map(lambda x: readimg_to_tensor(x))


In [ ]:
#output to CSV
VDlabel = np.array([])
img = next(iter(val_data.batch(10000)))
result = model.predict(img,use_multiprocessing = True)
test = parseResult(result)
val_file_name = pd.read_csv("./test/data01_test.csv")
val_file_name.join(pd.DataFrame(test,columns=["code"])).to_csv("data01_test.csv",index = False) 

In [10]:
#02
val_dir_data02 = os.path.abspath(os.getcwd())+"\\test\\data02_test"
#validation data no crop
val_dir = tf.data.Dataset.list_files(val_dir_data02+'*.jpg',shuffle=False) 
val_data02 = val_dir.map(lambda x: readimg_to_tensor(x))


In [11]:
#output to CSV
img = next(iter(val_data02.batch(10000)))
test = ensemble_chr(model_paths,img)
val_file_name = pd.read_csv("./dev/data02_dev.csv")
val_file_name.join(pd.DataFrame(test,columns=["code"])).to_csv("data02_test.csv",index = False) 

In [ ]:
#output to CSV
VDlabel = np.array([])
img = next(iter(val_data02.batch(10000)))
result = model.predict(img,use_multiprocessing = True)
val_dir = tf.data.Dataset.list_files(val_dir_data02+'*.jpg',shuffle=False) 
test = parseResult(result)
val_file_name = pd.read_csv("./dev/data02_dev.csv")
val_file_name.join(pd.DataFrame(test,columns=["code"])).to_csv("data02_dev.csv",index = False) 

In [ ]:
import tensorflow.keras.models as models

In [ ]:
#visualize
def show_model_vision(model,img):
    
    img_tensor = tf.expand_dims(img[np.random.randint(10000)],0)
    result = model(img_tensor)
    PredChr=''
    for i in range(6):
        PredChr+= (
            CLASS_NAMES[
            np.where(result[i].numpy()[0] == result[i].numpy()[0].max())[0][0]
            ])
    
    print("Predict character: "+ PredChr)
    # Extracts the outputs of the top 8 layers:
    layer_outputs = [layer.output for layer in model.layers[:] if hasattr(layer,"kernel_size") and getattr(layer,"kernel_size")==(3,3)]
    # Creates a model that will return these outputs, given the model input:
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
    # one array per layer activation
    activations = activation_model.predict(img_tensor)
    
    # These are the names of the layers, so can have them as part of our plot
    layer_names = []
    for layer in model.layers[:]:
        if hasattr(layer,"kernel_size") and getattr(layer,"kernel_size")==(3,3):
            layer_names.append(layer.name) 

    images_per_row = 3
    # Now let's display our feature maps
    k=1
    for layer_name, layer_activation in zip(layer_names, activations):
        for n in range(images_per_row):
            ax = plt.subplot(len(layer_names),images_per_row,k)
            plt.imshow(layer_activation[0, :, :,np.random.randint(layer_activation.shape[-1]) ], cmap='viridis')
            plt.title(layer_name)
            plt.axis('image')
            plt.tight_layout()
            k+=1
    plt.show()

In [ ]:
plt.rcParams['figure.figsize']=(30,30)

In [ ]:
show_model_vision(model,img)

# 紀錄

## data 01

Raiway_ini_and_WR_L0.4383.hdf5  95.47%

ResNet_small_4_mixSGD_.40.hdf5  93.74%

ResNet_small_4_mixSGD_.57587.hdf5 92.87%

ResNet_small_4_mixSGD_.4219.hdf5 92.68%

railway_2_wr1e-3.hdf5 97.24%

railway_2_wr_5e-3.hdf5 97.72%

railway_2_d12e-2.hdf5 97.53%

railway_2_b64_wr5e-3.hdf5 97.54%

railway_2_b256_wr5e-3.hdf5 97.74%

railway_2_b256_wr5e-3 (1).hdf5 97.74%

railway_2_b256_wr5e-31.hdf5 97.9%

railway_aug_d3e-1.hdf5 97.44%

railway_aug_d3e-1 copy.hdf5 97.49%



## data 02

railway_d2e-1.hdf5  94.87%

railway_d3e-1.hdf5  94.58%

## test 01

slyneGA_wr2.hdf5    98.11%

railway_2_b256_wr5e-31.hdf5 97.99%

slyneGA_wr2-1.hdf5  97.8%

slyneGA_wr2-2.hdf5  98.35%  / val:98.78%

slyneGA_wr2-3.hdf5  98.53%  / val:98.9%

slyneGA_GRU2.hdf5  98.64%  / val:98.888%  ***********

slyneGA_GRU.hdf5  98.51%  / val:98.888%

slyneGA_GRU3.hdf5 98.61% / val:98.84%

## test 02

1. 多數決

"railway_d2e-1.hdf5","CNN_GRU_3_fine2.hdf5","CNN_GRU_4.hdf5","CNN_GRU_5.hdf5","CNN+GRU.hdf5","CNN_GRU_2.hdf5" 

val: 97.46%     test:97.39 %  

